In [ ]:
import ee
import geemap as gee
import xarray as xr
import geemap

In [ ]:
!pip install xee
import xee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.1 MB/s eta 0:00:00


In [ ]:
ee.Authenticate()
ee.Initialize( project = 'ee-mussamahammed1', opt_url = 'https://earthengine-highvolume.googleapis.com')

In [ ]:
# ✅ Load Dynamic World LULC Dataset (2015–2021)
def get_year_image(y):
    year = ee.Number(y).int()
    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)

    image = ee.ImageCollection("GOOGLE/DYNAMICWORLD/V1") \
        .filterDate(start, end) \
        .median() \
        .select('label') \
        .set('year', year)
    return image

# ✅ Create ImageCollection for 2015–2021
years = ee.List.sequence(2015, 2021)
lulc_images = years.map(lambda y: get_year_image(y))
lulc_col = ee.ImageCollection.fromImages(lulc_images)

# ✅ Select Baseline Year (2021)
lulc2021 = ee.Image(lulc_col.filter(ee.Filter.eq('year', 2021)).first())

# ✅ Class codes
GRASS = 2
TREE = 1
SHRUB = 5

# ✅ Scenario 1: Grass + Trees → Shrub
condition1 = lulc2021.eq(GRASS).Or(lulc2021.eq(TREE))
scenario1 = lulc2021.where(condition1, SHRUB)

# ✅ Scenario 2: Half of Shrub → Grass
shrub_mask = lulc2021.eq(SHRUB)
random_mask = ee.Image.random().lt(0.5)
shrub_to_grass = shrub_mask.And(random_mask)
scenario2 = lulc2021.where(shrub_to_grass, GRASS)

# ✅ Scenario 3: All Shrub → Grass
scenario3 = lulc2021.where(lulc2021.eq(SHRUB), GRASS)

# ✅ Create an ImageCollection with all scenarios
all_scenarios = ee.ImageCollection.fromImages([
    lulc2021.set('scenario', 'Baseline 2021'),
    scenario1.set('scenario', 'Scenario 1'),
    scenario2.set('scenario', 'Scenario 2'),
    scenario3.set('scenario', 'Scenario 3')
])

print("All scenarios count:", all_scenarios.size().getInfo())


All scenarios count: 4


In [ ]:
fldas = ee.ImageCollection("NASA/FLDAS/NOAH01/C/GL/M/V001").select('SoilMoi00_10cm_tavg').filterDate('1982-01-01', '2014-12-31')

In [ ]:
# Define temporal aggregation function with mixed sum/mean
def temporal_collection_mixed(collection, start, count, interval, unit):
    seq = ee.List.sequence(0, count - 1)
    origin_date = ee.Date(start)

    def map_function(i):
        i = ee.Number(i)
        start_date = origin_date.advance(i.multiply(interval), unit)
        end_date = origin_date.advance(i.add(1).multiply(interval), unit)

        month_data = collection.filterDate(start_date, end_date)

        # Sum for precipitation
        pr_sum = month_data.select('pr').sum()

        # Mean for other variables
        others_mean = month_data.select(['tas', 'rlds', 'sfcWind', 'rsds', 'hurs']).mean()

        # Combine bands
        combined = pr_sum.addBands(others_mean) \
                         .set('system:time_start', start_date.millis()) \
                         .set('system:time_end', end_date.millis()) \
                         .set('date', start_date.format('YYYY-MM'))

        return combined

    return ee.ImageCollection(seq.map(map_function))


# Function to load and aggregate monthly data for multiple bands
def get_monthly_model_data(model_name):
    col = (ee.ImageCollection('NASA/GDDP-CMIP6')
           .filterDate('1982-01-01', '2014-12-31')
           .filter(ee.Filter.eq('model', model_name))
           .filter(ee.Filter.eq('scenario', 'historical'))
           .select('pr', 'tas', 'rlds', 'sfcWind', 'rsds', 'hurs'))
    return temporal_collection_mixed(col, '1982-01-01', 396, 1, 'month')


# List of model names
models = [
    'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 'EC-Earth3',
    'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL',
    'INM-CM4-8', 'INM-CM5-0', 'KACE-1-0-G'
]

# Load and prepare data for each model
monthly_data = [get_monthly_model_data(model).sort('system:time_start') for model in models]

# Convert each ImageCollection to a list
lists = [data.toList(data.size()) for data in monthly_data]

# Get size of one list (assuming all have same length)
size = lists[0].size()

# Function to compute mean across models for all bands
def compute_mean(i):
    i = ee.Number(i)
    images = [ee.Image(lst.get(i)) for lst in lists]
    sum_img = images[0]
    for img in images[1:]:
        sum_img = sum_img.add(img)
    mean_img = sum_img.divide(len(images)).rename([
        'mean_pr', 'mean_tas', 'mean_rlds', 'mean_sfcWind', 'mean_rsds', 'mean_hurs'
    ])
    return mean_img.set('system:time_start', images[0].get('system:time_start'))

# Combine into a mean ImageCollection
mean_monthly_data2 = ee.ImageCollection(ee.List.sequence(0, size.subtract(1)).map(compute_mean))

In [ ]:
# Join on system:time_start
filter_time_eq = ee.Filter.equals(leftField='system:time_start', rightField='system:time_start')
inner_join = ee.Join.inner()
joined = inner_join.apply(mean_monthly_data2, fldas, filter_time_eq)

# Merge images
def merge_images(pair):
    # DO NOT wrap with ee.Dictionary — it's already a dictionary-like object
    primary = ee.Image(pair.get('primary'))
    secondary = ee.Image(pair.get('secondary'))
    return primary.addBands(secondary)

# Map and return stacked collection
stacked_collection = ee.ImageCollection(joined.map(merge_images))

In [ ]:
def add_scenario3(img):
    return img.addBands(scenario3.rename('scenario3'))

stacked_with_scenario = stacked_collection.map(add_scenario3)

In [ ]:
roi = ([36.84427133765727, 3.3573318622713004,43.38113657203227, 7.501549141775435])

In [ ]:
# Load to xarray
ds = xr.open_dataset(
    stacked_with_scenario,
    engine="ee",
    crs="EPSG:4326",
    scale=0.1,
    geometry=roi
)

ds

<xarray.Dataset> Size: 34MB
Dimensions:              (time: 396, lon: 65, lat: 41)
Coordinates:
  * time                 (time) datetime64[ns] 3kB 1982-01-01 ... 2014-12-01
  * lon                  (lon) float64 520B 36.89 36.99 37.09 ... 43.19 43.29
  * lat                  (lat) float64 328B 3.407 3.507 3.607 ... 7.307 7.407
Data variables:
    mean_pr              (time, lon, lat) float32 4MB ...
    mean_tas             (time, lon, lat) float32 4MB ...
    mean_rlds            (time, lon, lat) float32 4MB ...
    mean_sfcWind         (time, lon, lat) float32 4MB ...
    mean_rsds            (time, lon, lat) float32 4MB ...
    mean_hurs            (time, lon, lat) float32 4MB ...
    SoilMoi00_10cm_tavg  (time, lon, lat) float32 4MB ...
    scenario3            (time, lon, lat) float32 4MB ...
Attributes:
    crs:      EPSG:4326

In [ ]:
df = ds.to_dataframe().dropna()

df

mean_pr    mean_tas   mean_rlds  mean_sfcWind  \
time       lon       lat                                                        
1982-01-01 36.894271 3.407332  0.000357  301.364838  374.429077      3.177507   
                     3.507332  0.000326  300.505768  370.673645      3.012533   
                     3.607332  0.000326  300.505768  370.673645      3.012533   
                     3.707332  0.000326  300.505768  370.673645      3.012533   
                     3.807332  0.000299  300.222687  369.369019      2.861456   
...                                 ...         ...         ...           ...   
2014-12-01 43.294271 7.007332  0.000067  298.117279  370.536804      2.550862   
                     7.107332  0.000067  298.117279  370.536804      2.550862   
                     7.207332  0.000067  298.117279  370.536804      2.550862   
                     7.307332  0.000059  297.201355  365.269531      2.618924   
                     7.407332  0.000059  297.201355  365.269531      2.618924   

                                mean_rsds  mean_hurs  SoilMoi00_10cm_tavg  \
time       lon       lat                                                    
1982-01-01 36.894271 3.407332  270.592651  50.339825             0.164703   
                     3.507332  269.167603  50.548820             0.165078   
                     3.607332  269.167603  50.548820             0.166346   
                     3.707332  269.167603  50.548820             0.167031   
                     3.807332  267.745148  50.749165             0.167412   
...                                   ...        ...                  ...   
2014-12-01 43.294271 7.007332  235.588531  48.995857             0.182852   
                     7.107332  235.588531  48.995857             0.181655   
                     7.207332  235.588531  48.995857             0.177792   
                     7.307332  236.970001  48.355278             0.173464   
                     7.407332  236.970001  48.355278             0.171077   

                               scenario3  
time       lon       lat                  
1982-01-01 36.894271 3.407332        7.0  
                     3.507332        7.0  
                     3.607332        7.0  
                     3.707332        4.0  
                     3.807332        2.0  
...                                  ...  
2014-12-01 43.294271 7.007332        2.0  
                     7.107332        2.0  
                     7.207332        2.0  
                     7.307332        4.0  
                     7.407332        2.0  

[1055340 rows x 8 columns]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

In [ ]:
# Define predictors and target
X = df[['mean_pr', 'mean_tas', 'mean_rlds',
        'mean_sfcWind', 'mean_rsds', 'mean_hurs', 'scenario3']]
y = df['SoilMoi00_10cm_tavg']

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Train Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

In [ ]:
# ✅ Evaluation Metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"MAE : {mae:.4f}")
print(f"R²   : {r2:.4f}")

RMSE: 0.0158
MAE : 0.0098
R²   : 0.9509


In [ ]:
#model = rf_model

In [ ]:
df['predicted_sm'] = model.predict(X)

df

mean_pr    mean_tas   mean_rlds  mean_sfcWind  \
time       lon       lat                                                        
1982-01-01 36.894271 3.407332  0.000357  301.364838  374.429077      3.177507   
                     3.507332  0.000326  300.505768  370.673645      3.012533   
                     3.607332  0.000326  300.505768  370.673645      3.012533   
                     3.707332  0.000326  300.505768  370.673645      3.012533   
                     3.807332  0.000299  300.222687  369.369019      2.861456   
...                                 ...         ...         ...           ...   
2014-12-01 43.294271 7.007332  0.000067  298.117279  370.536804      2.550862   
                     7.107332  0.000067  298.117279  370.536804      2.550862   
                     7.207332  0.000067  298.117279  370.536804      2.550862   
                     7.307332  0.000059  297.201355  365.269531      2.618924   
                     7.407332  0.000059  297.201355  365.269531      2.618924   

                                mean_rsds  mean_hurs  SoilMoi00_10cm_tavg  \
time       lon       lat                                                    
1982-01-01 36.894271 3.407332  270.592651  50.339825             0.164703   
                     3.507332  269.167603  50.548820             0.165078   
                     3.607332  269.167603  50.548820             0.166346   
                     3.707332  269.167603  50.548820             0.167031   
                     3.807332  267.745148  50.749165             0.167412   
...                                   ...        ...                  ...   
2014-12-01 43.294271 7.007332  235.588531  48.995857             0.182852   
                     7.107332  235.588531  48.995857             0.181655   
                     7.207332  235.588531  48.995857             0.177792   
                     7.307332  236.970001  48.355278             0.173464   
                     7.407332  236.970001  48.355278             0.171077   

                               scenario3  predicted_sm  
time       lon       lat                                
1982-01-01 36.894271 3.407332        7.0      0.172592  
                     3.507332        7.0      0.165829  
                     3.607332        7.0      0.165829  
                     3.707332        4.0      0.166609  
                     3.807332        2.0      0.168047  
...                                  ...           ...  
2014-12-01 43.294271 7.007332        2.0      0.181805  
                     7.107332        2.0      0.181805  
                     7.207332        2.0      0.181805  
                     7.307332        4.0      0.174570  
                     7.407332        2.0      0.174069  

[1055340 rows x 9 columns]

In [ ]:
# Define temporal aggregation function with mixed sum/mean
def temporal_collection_mixed(collection, start, count, interval, unit):
    seq = ee.List.sequence(0, count - 1)
    origin_date = ee.Date(start)

    def map_function(i):
        i = ee.Number(i)
        start_date = origin_date.advance(i.multiply(interval), unit)
        end_date = origin_date.advance(i.add(1).multiply(interval), unit)

        month_data = collection.filterDate(start_date, end_date)

        # Sum for precipitation
        pr_sum = month_data.select('pr').sum()

        # Mean for other variables
        others_mean = month_data.select(['tas', 'rlds', 'sfcWind', 'rsds', 'hurs']).mean()

        # Combine bands
        combined = pr_sum.addBands(others_mean) \
                         .set('system:time_start', start_date.millis()) \
                         .set('system:time_end', end_date.millis()) \
                         .set('date', start_date.format('YYYY-MM'))

        return combined

    return ee.ImageCollection(seq.map(map_function))


# Function to load and aggregate monthly data for multiple bands
def get_monthly_model_data(model_name):
    col = (ee.ImageCollection('NASA/GDDP-CMIP6')
           .filterDate('2015-01-01', '2100-12-31')
           .filter(ee.Filter.eq('model', model_name))
           .filter(ee.Filter.eq('scenario', 'ssp245'))
           .select('pr', 'tas', 'rlds', 'sfcWind', 'rsds', 'hurs'))
    return temporal_collection_mixed(col, '2015-01-01', 1020, 1, 'month')


# List of model names
models = [
    'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 'EC-Earth3',
    'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL',
    'INM-CM4-8', 'INM-CM5-0', 'KACE-1-0-G'
]

# Load and prepare data for each model
monthly_data = [get_monthly_model_data(model).sort('system:time_start') for model in models]

# Convert each ImageCollection to a list
lists = [data.toList(data.size()) for data in monthly_data]

# Get size of one list (assuming all have same length)
size = lists[0].size()

# Function to compute mean across models for all bands
def compute_mean(i):
    i = ee.Number(i)
    images = [ee.Image(lst.get(i)) for lst in lists]
    sum_img = images[0]
    for img in images[1:]:
        sum_img = sum_img.add(img)
    mean_img = sum_img.divide(len(images)).rename([
        'mean_pr', 'mean_tas', 'mean_rlds', 'mean_sfcWind', 'mean_rsds', 'mean_hurs'
    ])
    return mean_img.set('system:time_start', images[0].get('system:time_start'))

# Combine into a mean ImageCollection
mean_monthly_data20 = ee.ImageCollection(ee.List.sequence(0, size.subtract(1)).map(compute_mean))

In [ ]:
def add_scenario3(img):
    return img.addBands(scenario3.rename('scenario3'))

stacked_with_scenario1 = mean_monthly_data20.map(add_scenario3)

In [ ]:
dt = xr.open_dataset(
    stacked_with_scenario1,
    engine = 'ee',
    crs = 'EPSG:4326',
    scale = 0.1,
    geometry = roi
)

dt

<xarray.Dataset> Size: 76MB
Dimensions:       (time: 1020, lon: 65, lat: 41)
Coordinates:
  * time          (time) datetime64[ns] 8kB 2015-01-01 2015-02-01 ... 2099-12-01
  * lon           (lon) float64 520B 36.89 36.99 37.09 ... 43.09 43.19 43.29
  * lat           (lat) float64 328B 3.407 3.507 3.607 ... 7.207 7.307 7.407
Data variables:
    mean_pr       (time, lon, lat) float32 11MB ...
    mean_tas      (time, lon, lat) float32 11MB ...
    mean_rlds     (time, lon, lat) float32 11MB ...
    mean_sfcWind  (time, lon, lat) float32 11MB ...
    mean_rsds     (time, lon, lat) float32 11MB ...
    mean_hurs     (time, lon, lat) float32 11MB ...
    scenario3     (time, lon, lat) float32 11MB ...
Attributes:
    crs:      EPSG:4326

In [ ]:
dy = dt.to_dataframe().dropna()
dy

mean_pr    mean_tas   mean_rlds  mean_sfcWind  \
time       lon       lat                                                        
2015-01-01 36.894271 3.407332  0.000242  301.912354  383.265015      3.000198   
                     3.507332  0.000225  301.055176  379.595367      2.864615   
                     3.607332  0.000225  301.055176  379.595367      2.864615   
                     3.707332  0.000225  301.055176  379.595367      2.864615   
                     3.807332  0.000217  300.766663  378.301422      2.723913   
...                                 ...         ...         ...           ...   
2099-12-01 43.294271 7.007332  0.000174  300.350616  383.046326      2.612536   
                     7.107332  0.000174  300.350616  383.046326      2.612536   
                     7.207332  0.000174  300.350616  383.046326      2.612536   
                     7.307332  0.000186  299.413940  377.646515      2.660689   
                     7.407332  0.000186  299.413940  377.646515      2.660689   

                                mean_rsds  mean_hurs  scenario3  
time       lon       lat                                         
2015-01-01 36.894271 3.407332  260.476440  52.410080        7.0  
                     3.507332  258.907013  52.560261        7.0  
                     3.607332  258.907013  52.560261        7.0  
                     3.707332  258.907013  52.560261        4.0  
                     3.807332  257.526642  52.728371        2.0  
...                                   ...        ...        ...  
2099-12-01 43.294271 7.007332  231.821091  51.116966        2.0  
                     7.107332  231.821091  51.116966        2.0  
                     7.207332  231.821091  51.116966        2.0  
                     7.307332  233.509552  50.353699        4.0  
                     7.407332  233.509552  50.353699        2.0  

[2718300 rows x 7 columns]

In [ ]:
dy['sm1'] = model.predict(dy[['mean_pr', 'mean_tas', 'mean_rlds', 'mean_sfcWind', 'mean_rsds', 'mean_hurs','scenario3']]) # Changed column names to match those in df0
dy

mean_pr    mean_tas   mean_rlds  mean_sfcWind  \
time       lon       lat                                                        
2015-01-01 36.894271 3.407332  0.000242  301.912354  383.265015      3.000198   
                     3.507332  0.000225  301.055176  379.595367      2.864615   
                     3.607332  0.000225  301.055176  379.595367      2.864615   
                     3.707332  0.000225  301.055176  379.595367      2.864615   
                     3.807332  0.000217  300.766663  378.301422      2.723913   
...                                 ...         ...         ...           ...   
2099-12-01 43.294271 7.007332  0.000174  300.350616  383.046326      2.612536   
                     7.107332  0.000174  300.350616  383.046326      2.612536   
                     7.207332  0.000174  300.350616  383.046326      2.612536   
                     7.307332  0.000186  299.413940  377.646515      2.660689   
                     7.407332  0.000186  299.413940  377.646515      2.660689   

                                mean_rsds  mean_hurs  scenario3       sm1  
time       lon       lat                                                   
2015-01-01 36.894271 3.407332  260.476440  52.410080        7.0  0.169420  
                     3.507332  258.907013  52.560261        7.0  0.182154  
                     3.607332  258.907013  52.560261        7.0  0.182154  
                     3.707332  258.907013  52.560261        4.0  0.182900  
                     3.807332  257.526642  52.728371        2.0  0.178043  
...                                   ...        ...        ...       ...  
2099-12-01 43.294271 7.007332  231.821091  51.116966        2.0  0.211849  
                     7.107332  231.821091  51.116966        2.0  0.211849  
                     7.207332  231.821091  51.116966        2.0  0.211849  
                     7.307332  233.509552  50.353699        4.0  0.186637  
                     7.407332  233.509552  50.353699        2.0  0.187768  

[2718300 rows x 8 columns]

In [ ]:
# Define temporal aggregation function with mixed sum/mean
def temporal_collection_mixed(collection, start, count, interval, unit):
    seq = ee.List.sequence(0, count - 1)
    origin_date = ee.Date(start)

    def map_function(i):
        i = ee.Number(i)
        start_date = origin_date.advance(i.multiply(interval), unit)
        end_date = origin_date.advance(i.add(1).multiply(interval), unit)

        month_data = collection.filterDate(start_date, end_date)

        # Sum for precipitation
        pr_sum = month_data.select('pr').sum()

        # Mean for other variables
        others_mean = month_data.select(['tas', 'rlds', 'sfcWind', 'rsds', 'hurs']).mean()

        # Combine bands
        combined = pr_sum.addBands(others_mean) \
                         .set('system:time_start', start_date.millis()) \
                         .set('system:time_end', end_date.millis()) \
                         .set('date', start_date.format('YYYY-MM'))

        return combined

    return ee.ImageCollection(seq.map(map_function))


# Function to load and aggregate monthly data for multiple bands
def get_monthly_model_data(model_name):
    col = (ee.ImageCollection('NASA/GDDP-CMIP6')
           .filterDate('2015-01-01', '2100-12-31')
           .filter(ee.Filter.eq('model', model_name))
           .filter(ee.Filter.eq('scenario', 'ssp585'))
           .select('pr', 'tas', 'rlds', 'sfcWind', 'rsds', 'hurs'))
    return temporal_collection_mixed(col, '2015-01-01', 1020, 1, 'month')


# List of model names
models = [
    'CNRM-CM6-1', 'CNRM-ESM2-1', 'CanESM5', 'EC-Earth3',
    'FGOALS-g3', 'GISS-E2-1-G', 'HadGEM3-GC31-LL',
    'INM-CM4-8', 'INM-CM5-0', 'KACE-1-0-G'
]

# Load and prepare data for each model
monthly_data = [get_monthly_model_data(model).sort('system:time_start') for model in models]

# Convert each ImageCollection to a list
lists = [data.toList(data.size()) for data in monthly_data]

# Get size of one list (assuming all have same length)
size = lists[0].size()

# Function to compute mean across models for all bands
def compute_mean(i):
    i = ee.Number(i)
    images = [ee.Image(lst.get(i)) for lst in lists]
    sum_img = images[0]
    for img in images[1:]:
        sum_img = sum_img.add(img)
    mean_img = sum_img.divide(len(images)).rename([
        'mean_pr', 'mean_tas', 'mean_rlds', 'mean_sfcWind', 'mean_rsds', 'mean_hurs'
    ])
    return mean_img.set('system:time_start', images[0].get('system:time_start'))

# Combine into a mean ImageCollection
mean_monthly_data21 = ee.ImageCollection(ee.List.sequence(0, size.subtract(1)).map(compute_mean))

In [ ]:
def add_scenario3(img):
    return img.addBands(scenario3.rename('scenario3'))

stacked_with_scenario2 = mean_monthly_data21.map(add_scenario3)

In [ ]:
dt1 = xr.open_dataset(
   stacked_with_scenario2,
    engine = 'ee',
    crs = 'EPSG:4326',
    scale = 0.1,
    geometry = roi
)

dt1

<xarray.Dataset> Size: 76MB
Dimensions:       (time: 1020, lon: 65, lat: 41)
Coordinates:
  * time          (time) datetime64[ns] 8kB 2015-01-01 2015-02-01 ... 2099-12-01
  * lon           (lon) float64 520B 36.89 36.99 37.09 ... 43.09 43.19 43.29
  * lat           (lat) float64 328B 3.407 3.507 3.607 ... 7.207 7.307 7.407
Data variables:
    mean_pr       (time, lon, lat) float32 11MB ...
    mean_tas      (time, lon, lat) float32 11MB ...
    mean_rlds     (time, lon, lat) float32 11MB ...
    mean_sfcWind  (time, lon, lat) float32 11MB ...
    mean_rsds     (time, lon, lat) float32 11MB ...
    mean_hurs     (time, lon, lat) float32 11MB ...
    scenario3     (time, lon, lat) float32 11MB ...
Attributes:
    crs:      EPSG:4326

In [ ]:
dt = dt1.to_dataframe().dropna()
dt

mean_pr    mean_tas   mean_rlds  mean_sfcWind  \
time       lon       lat                                                        
2015-01-01 36.894271 3.407332  0.000162  301.979980  382.193848      3.054869   
                     3.507332  0.000151  301.125458  378.472687      2.911800   
                     3.607332  0.000151  301.125458  378.472687      2.911800   
                     3.707332  0.000151  301.125458  378.472687      2.911800   
                     3.807332  0.000143  300.844879  377.121948      2.764918   
...                                 ...         ...         ...           ...   
2099-12-01 43.294271 7.007332  0.000309  302.731659  402.650421      2.244829   
                     7.107332  0.000309  302.731659  402.650421      2.244829   
                     7.207332  0.000309  302.731659  402.650421      2.244829   
                     7.307332  0.000292  301.866333  397.027374      2.290747   
                     7.407332  0.000292  301.866333  397.027374      2.290747   

                                mean_rsds  mean_hurs  scenario3  
time       lon       lat                                         
2015-01-01 36.894271 3.407332  263.912201  50.911446        7.0  
                     3.507332  262.612610  51.025215        7.0  
                     3.607332  262.612610  51.025215        7.0  
                     3.707332  262.612610  51.025215        4.0  
                     3.807332  261.411224  51.167877        2.0  
...                                   ...        ...        ...  
2099-12-01 43.294271 7.007332  224.003845  56.466118        2.0  
                     7.107332  224.003845  56.466118        2.0  
                     7.207332  224.003845  56.466118        2.0  
                     7.307332  225.891876  55.359032        4.0  
                     7.407332  225.891876  55.359032        2.0  

[2718300 rows x 7 columns]

In [ ]:
dt['sm2'] = model.predict(dt[['mean_pr', 'mean_tas', 'mean_rlds', 'mean_sfcWind', 'mean_rsds', 'mean_hurs','scenario3']]) # Changed column names to match those in df0
dt

mean_pr    mean_tas   mean_rlds  mean_sfcWind  \
time       lon       lat                                                        
2015-01-01 36.894271 3.407332  0.000162  301.979980  382.193848      3.054869   
                     3.507332  0.000151  301.125458  378.472687      2.911800   
                     3.607332  0.000151  301.125458  378.472687      2.911800   
                     3.707332  0.000151  301.125458  378.472687      2.911800   
                     3.807332  0.000143  300.844879  377.121948      2.764918   
...                                 ...         ...         ...           ...   
2099-12-01 43.294271 7.007332  0.000309  302.731659  402.650421      2.244829   
                     7.107332  0.000309  302.731659  402.650421      2.244829   
                     7.207332  0.000309  302.731659  402.650421      2.244829   
                     7.307332  0.000292  301.866333  397.027374      2.290747   
                     7.407332  0.000292  301.866333  397.027374      2.290747   

                                mean_rsds  mean_hurs  scenario3       sm2  
time       lon       lat                                                   
2015-01-01 36.894271 3.407332  263.912201  50.911446        7.0  0.174910  
                     3.507332  262.612610  51.025215        7.0  0.196152  
                     3.607332  262.612610  51.025215        7.0  0.196152  
                     3.707332  262.612610  51.025215        4.0  0.196835  
                     3.807332  261.411224  51.167877        2.0  0.197781  
...                                   ...        ...        ...       ...  
2099-12-01 43.294271 7.007332  224.003845  56.466118        2.0  0.257158  
                     7.107332  224.003845  56.466118        2.0  0.257158  
                     7.207332  224.003845  56.466118        2.0  0.257158  
                     7.307332  225.891876  55.359032        4.0  0.214070  
                     7.407332  225.891876  55.359032        2.0  0.214283  

[2718300 rows x 8 columns]

In [ ]:
import pandas as pd

# ===== Process 1982–2014 dataset =====
if not isinstance(df.index.get_level_values('time'), pd.DatetimeIndex):
    df.index = df.index.set_levels(pd.to_datetime(df.index.levels[0]), level='time')

df_reset = df.reset_index()

monthly_means_df = (
    df_reset.groupby(df_reset['time'].dt.to_period('M'))
    [['mean_pr', 'mean_tas', 'predicted_sm']]
    .mean()
    .reset_index()
)

monthly_means_df['time'] = monthly_means_df['time'].dt.to_timestamp()
monthly_means_df.rename(columns={'predicted_sm': 'sm'}, inplace=True)

# ===== Process 2015–2100 SSP245 dataset =====
if not isinstance(dy.index.get_level_values('time'), pd.DatetimeIndex):
    dy.index = dy.index.set_levels(pd.to_datetime(dy.index.levels[0]), level='time')

dy_reset = dy.reset_index()

monthly_means_dy = (
    dy_reset.groupby(dy_reset['time'].dt.to_period('M'))
    [['mean_pr', 'mean_tas', 'sm1']]
    .mean()
    .reset_index()
)

monthly_means_dy['time'] = monthly_means_dy['time'].dt.to_timestamp()
monthly_means_dy.rename(columns={'sm1': 'sm'}, inplace=True)

# ===== Merge both datasets =====
merged_monthly_means = pd.concat([monthly_means_df, monthly_means_dy], ignore_index=True)
merged_monthly_means.sort_values(by='time', inplace=True)

# ===== Export to CSV =====
merged_monthly_means.to_csv("monthly_means_1982_2100_ssp245_SM_scenario3.csv", index=False)

print("✅ Combined monthly means saved to monthly_means_1982_2100_ssp245_SM_scenario3.csv")

✅ Combined monthly means saved to monthly_means_1982_2100_ssp245_SM_scenario3.csv


In [ ]:
import pandas as pd

# ===== Process 1982–2014 dataset =====
if not isinstance(df.index.get_level_values('time'), pd.DatetimeIndex):
    df.index = df.index.set_levels(pd.to_datetime(df.index.levels[0]), level='time')

df_reset = df.reset_index()

monthly_means_df = (
    df_reset.groupby(df_reset['time'].dt.to_period('M'))
    [['mean_pr', 'mean_tas', 'predicted_sm']]
    .mean()
    .reset_index()
)

monthly_means_df['time'] = monthly_means_df['time'].dt.to_timestamp()
monthly_means_df.rename(columns={'predicted_sm': 'sm'}, inplace=True)

# ===== Process 2015–2100 SSP585 dataset =====
if not isinstance(dt.index.get_level_values('time'), pd.DatetimeIndex):
    dt.index = dt.index.set_levels(pd.to_datetime(dt.index.levels[0]), level='time')

dt_reset = dt.reset_index()

monthly_means_dt = (
    dt_reset.groupby(dt_reset['time'].dt.to_period('M'))
    [['mean_pr', 'mean_tas', 'sm2']]
    .mean()
    .reset_index()
)

monthly_means_dt['time'] = monthly_means_dt['time'].dt.to_timestamp()
monthly_means_dt.rename(columns={'sm2': 'sm'}, inplace=True)

# ===== Merge both datasets =====
merged_monthly_means = pd.concat([monthly_means_df, monthly_means_dt], ignore_index=True)
merged_monthly_means.sort_values(by='time', inplace=True)

# ===== Export to CSV =====
merged_monthly_means.to_csv("monthly_means_1982_2100_ssp585_SM_scenario3.csv", index=False)

print("✅ Combined monthly means saved to monthly_means_1982_2100_ssp585_SM_scenario3.csv")

✅ Combined monthly means saved to monthly_means_1982_2100_ssp585_SM_scenario3.csv
